<a href="https://colab.research.google.com/github/raghavendranhp/Financial_Risk_Detection/blob/main/Pre_process_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
from pprint import PrettyPrinter
from datetime import datetime, time

#Data visualization libraries
import seaborn as sns
sns.set()
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.style as style
import itertools
%matplotlib inline

#Ignore FutureWarnings to avoid clutter in the output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#Initialize PrettyPrinter for neat printing
pp = PrettyPrinter(indent=2)

In [2]:
#Set the maximum number of displayed columns for Pandas DataFrames to 'None' (display all columns)
pd.set_option('display.max_columns', None)

#Set the maximum number of displayed rows for Pandas DataFrames to 200
pd.set_option('display.max_rows', 200)

#Set the maximum column width to 'None' (display complete content)
pd.set_option('display.max_colwidth', None)

#Set the maximum width of the display in characters
pd.set_option('display.width', 1000)

#Enable the display of DataFrames with expanded representation
#This means if the DataFrame has too many columns to fit horizontally within the display,
#it will display it across multiple lines instead of compressing it into fewer lines.
pd.set_option('display.expand_frame_repr', True)


In [3]:
df_column_description_org=pd.read_excel(r"/content/drive/MyDrive/Dataset/Financial Risk Analysis/columns_description.xlsx")
df_app_data_org=pd.read_csv(r"/content/drive/MyDrive/Dataset/Financial Risk Analysis/application_data.csv")
df_pre_app_org=pd.read_csv(r"/content/drive/MyDrive/Dataset/Financial Risk Analysis/previous_application.csv")

In [4]:
#Make a copy of the original DataFrame to work with
df_app_data=df_app_data_org.copy()
df_pre_app=df_pre_app_org.copy()
df_column_description=df_column_description_org.copy()

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:2px; margin-bottom: 0em; text-align: center; font-size: 20px;border-style: solid;border-color: dark green;"> Previous Application Data</div>


In [5]:
df_pre_app.shape

(1670214, 37)

In [6]:
df_pre_app.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [7]:
df_pre_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_pre_app.describe()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
count,1.670214e+06,1.670214e+06,1.297979e+06,1.670214e+06,1.670213e+06,7.743700e+05,1.284699e+06,1.670214e+06,1.670214e+06,774370.000000,5951.000000,5951.000000,1.670214e+06,1.670214e+06,1.297984e+06,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000
mean,1.923089e+06,2.783572e+05,1.595512e+04,1.752339e+05,1.961140e+05,6.697402e+03,2.278473e+05,1.248418e+01,9.964675e-01,0.079637,0.188357,0.773503,-8.806797e+02,3.139511e+02,1.605408e+01,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838,0.332570
std,5.325980e+05,1.028148e+05,1.478214e+04,2.927798e+05,3.185746e+05,2.092150e+04,3.153966e+05,3.334028e+00,5.932963e-02,0.107823,0.087671,0.100879,7.790997e+02,7.127443e+03,1.456729e+01,88916.115833,72444.869708,106857.034789,149647.415123,153303.516729,0.471134
min,1.000001e+06,1.000010e+05,0.000000e+00,0.000000e+00,0.000000e+00,-9.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,-0.000015,0.034781,0.373150,-2.922000e+03,-1.000000e+00,0.000000e+00,-2922.000000,-2892.000000,-2801.000000,-2889.000000,-2874.000000,0.000000
25%,1.461857e+06,1.893290e+05,6.321780e+03,1.872000e+04,2.416050e+04,0.000000e+00,5.084100e+04,1.000000e+01,1.000000e+00,0.000000,0.160716,0.715645,-1.300000e+03,-1.000000e+00,6.000000e+00,365243.000000,-1628.000000,-1242.000000,-1314.000000,-1270.000000,0.000000
50%,1.923110e+06,2.787145e+05,1.125000e+04,7.104600e+04,8.054100e+04,1.638000e+03,1.123200e+05,1.200000e+01,1.000000e+00,0.051605,0.189122,0.835095,-5.810000e+02,3.000000e+00,1.200000e+01,365243.000000,-831.000000,-361.000000,-537.000000,-499.000000,0.000000
75%,2.384280e+06,3.675140e+05,2.065842e+04,1.803600e+05,2.164185e+05,7.740000e+03,2.340000e+05,1.500000e+01,1.000000e+00,0.108909,0.193330,0.852537,-2.800000e+02,8.200000e+01,2.400000e+01,365243.000000,-411.000000,129.000000,-74.000000,-44.000000,1.000000
max,2.845382e+06,4.562550e+05,4.180581e+05,6.905160e+06,6.905160e+06,3.060045e+06,6.905160e+06,2.300000e+01,1.000000e+00,1.000000,1.000000,1.000000,-1.000000e+00,4.000000e+06,8.400000e+01,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000,1.000000


In [9]:
#Display descriptive statistics for categorical (object) columns in the DataFrame
df_pre_app.describe(include="O")

,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
count,1670214,1670214,1670214,1670214,1670214,1670214,1670214,849809,1670214,1670214,1670214,1670214,1670214,1670214,1670214,1669868
unique,4,7,2,25,4,4,9,7,4,28,5,3,8,11,5,17
top,Cash loans,TUESDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,POS,XNA,Credit and cash offices,XNA,XNA,Cash
freq,747553,255118,1661739,922661,1036781,1033552,1353093,508970,1231261,950809,691011,1063666,719968,855720,517215,285990


In [10]:
#Get the names of categorical columns in the DataFrame
categorical_columns_pre=df_pre_app.select_dtypes(include='O').columns
#Get the names of numerical columns in the DataFrame
numerical_columns_pre=df_pre_app.select_dtypes(include='number').columns

In [11]:
#Iterate over each categorical column in the DataFrame
for i in categorical_columns_pre:
    #Print the unique values of the current categorical column
    print('Unique values of', i, set(df_pre_app[i]))
    #Print the count of each unique value in the current categorical column
    print(df_pre_app[i].value_counts())
    #Print a separator line for better readability
    print('-' * 50)

Unique values of NAME_CONTRACT_TYPE {'Revolving loans', 'Cash loans', 'XNA', 'Consumer loans'}
Cash loans         747553
Consumer loans     729151
Revolving loans    193164
XNA                   346
Name: NAME_CONTRACT_TYPE, dtype: int64
--------------------------------------------------
Unique values of WEEKDAY_APPR_PROCESS_START {'SUNDAY', 'FRIDAY', 'TUESDAY', 'THURSDAY', 'WEDNESDAY', 'SATURDAY', 'MONDAY'}
TUESDAY      255118
WEDNESDAY    255010
MONDAY       253557
FRIDAY       252048
THURSDAY     249099
SATURDAY     240631
SUNDAY       164751
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64
--------------------------------------------------
Unique values of FLAG_LAST_APPL_PER_CONTRACT {'Y', 'N'}
Y    1661739
N       8475
Name: FLAG_LAST_APPL_PER_CONTRACT, dtype: int64
--------------------------------------------------
Unique values of NAME_CASH_LOAN_PURPOSE {'Buying a garage', 'Business development', 'Wedding / gift / holiday', 'Furniture', 'Buying a used car', 'Other', 'Car repairs',

In [12]:
df_pre_app.isnull().sum()

SK_ID_PREV                           0
SK_ID_CURR                           0
NAME_CONTRACT_TYPE                   0
AMT_ANNUITY                     372235
AMT_APPLICATION                      0
AMT_CREDIT                           1
AMT_DOWN_PAYMENT                895844
AMT_GOODS_PRICE                 385515
WEEKDAY_APPR_PROCESS_START           0
HOUR_APPR_PROCESS_START              0
FLAG_LAST_APPL_PER_CONTRACT          0
NFLAG_LAST_APPL_IN_DAY               0
RATE_DOWN_PAYMENT               895844
RATE_INTEREST_PRIMARY          1664263
RATE_INTEREST_PRIVILEGED       1664263
NAME_CASH_LOAN_PURPOSE               0
NAME_CONTRACT_STATUS                 0
DAYS_DECISION                        0
NAME_PAYMENT_TYPE                    0
CODE_REJECT_REASON                   0
NAME_TYPE_SUITE                 820405
NAME_CLIENT_TYPE                     0
NAME_GOODS_CATEGORY                  0
NAME_PORTFOLIO                       0
NAME_PRODUCT_TYPE                    0
CHANNEL_TYPE             

In [13]:
print(df_pre_app[df_pre_app['AMT_CREDIT'].isnull()])
df_pre_app['AMT_CREDIT'] = df_pre_app['AMT_CREDIT'].fillna(0)

         SK_ID_PREV  SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE WEEKDAY_APPR_PROCESS_START  HOUR_APPR_PROCESS_START FLAG_LAST_APPL_PER_CONTRACT  NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  RATE_INTEREST_PRIMARY  RATE_INTEREST_PRIVILEGED NAME_CASH_LOAN_PURPOSE NAME_CONTRACT_STATUS  DAYS_DECISION NAME_PAYMENT_TYPE CODE_REJECT_REASON NAME_TYPE_SUITE NAME_CLIENT_TYPE NAME_GOODS_CATEGORY NAME_PORTFOLIO NAME_PRODUCT_TYPE  CHANNEL_TYPE  SELLERPLACE_AREA NAME_SELLER_INDUSTRY  CNT_PAYMENT NAME_YIELD_GROUP PRODUCT_COMBINATION  DAYS_FIRST_DRAWING  DAYS_FIRST_DUE  DAYS_LAST_DUE_1ST_VERSION  DAYS_LAST_DUE  DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL
1127152     2204450      438387    Revolving loans          0.0              0.0         NaN               NaN              NaN                     FRIDAY                       10                           Y                       1                NaN                    NaN                   

In [14]:
print(df_pre_app[df_pre_app['AMT_ANNUITY'].isnull()])

         SK_ID_PREV  SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE WEEKDAY_APPR_PROCESS_START  HOUR_APPR_PROCESS_START FLAG_LAST_APPL_PER_CONTRACT  NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  RATE_INTEREST_PRIMARY  RATE_INTEREST_PRIVILEGED NAME_CASH_LOAN_PURPOSE NAME_CONTRACT_STATUS  DAYS_DECISION      NAME_PAYMENT_TYPE CODE_REJECT_REASON NAME_TYPE_SUITE NAME_CLIENT_TYPE NAME_GOODS_CATEGORY NAME_PORTFOLIO NAME_PRODUCT_TYPE             CHANNEL_TYPE  SELLERPLACE_AREA NAME_SELLER_INDUSTRY  CNT_PAYMENT NAME_YIELD_GROUP       PRODUCT_COMBINATION  DAYS_FIRST_DRAWING  DAYS_FIRST_DUE  DAYS_LAST_DUE_1ST_VERSION  DAYS_LAST_DUE  DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL
6           2315218      175704         Cash loans          NaN              0.0         0.0               NaN              NaN                    TUESDAY                       11                           Y                       1                NaN                    

In [15]:
df_pre_app.loc[df_pre_app['NAME_CONTRACT_STATUS'] != 'Approved', 'AMT_ANNUITY'] = -1
df_pre_app.loc[ (df_pre_app['AMT_ANNUITY'] == 0),'AMT_ANNUITY'] = df_pre_app.loc[(df_pre_app['AMT_ANNUITY'] == 0), 'AMT_CREDIT'] / 12
df_pre_app.loc[df_pre_app['AMT_ANNUITY'].isnull(), 'AMT_ANNUITY'] = df_pre_app.loc[df_pre_app['AMT_ANNUITY'].isnull(),
                                                                                   'AMT_CREDIT'] / 12

In [16]:
df_pre_app['AMT_ANNUITY'].describe()

count    1.670214e+06
mean     9.137567e+03
std      1.301109e+04
min     -1.000000e+00
25%     -1.000000e+00
50%      5.126422e+03
75%      1.271357e+04
max      3.938687e+05
Name: AMT_ANNUITY, dtype: float64

In [17]:
#Define the bins
bins = [-2, 0, 500, 2000, 5000, 10000, 15000, 20000, float('inf')]
#Define labels for each bin
labels = ['NOT Approved', '0-500', '500-2000', '2000-5000', '5000-10000', '10000-15000', '15000-20000', '20000+']
#Create a new column with bin labels
df_pre_app['AMT_ANNUITY_RANGE'] = pd.cut(df_pre_app['AMT_ANNUITY'], bins=bins, labels=labels)
#Display the counts in each bin
print(df_pre_app['AMT_ANNUITY_RANGE'].value_counts())

NOT Approved    634985
5000-10000      305917
20000+          232813
10000-15000     197343
2000-5000       185227
15000-20000     106700
500-2000          7229
0-500                0
Name: AMT_ANNUITY_RANGE, dtype: int64


In [18]:
"""Set AMT_APPLICATION when there is zero in amt
df_pre_app.loc[(df_pre_app['NAME_CONTRACT_STATUS'] != 'Approved') &
 (df_pre_app['AMT_APPLICATION'] == 0), 'AMT_APPLICATION'] = -1"""

"Set AMT_APPLICATION when there is zero in amt\ndf_pre_app.loc[(df_pre_app['NAME_CONTRACT_STATUS'] != 'Approved') & \n (df_pre_app['AMT_APPLICATION'] == 0), 'AMT_APPLICATION'] = -1"

In [19]:
print(df_pre_app[df_pre_app['AMT_APPLICATION'].isnull()])

Empty DataFrame
Columns: [SK_ID_PREV, SK_ID_CURR, NAME_CONTRACT_TYPE, AMT_ANNUITY, AMT_APPLICATION, AMT_CREDIT, AMT_DOWN_PAYMENT, AMT_GOODS_PRICE, WEEKDAY_APPR_PROCESS_START, HOUR_APPR_PROCESS_START, FLAG_LAST_APPL_PER_CONTRACT, NFLAG_LAST_APPL_IN_DAY, RATE_DOWN_PAYMENT, RATE_INTEREST_PRIMARY, RATE_INTEREST_PRIVILEGED, NAME_CASH_LOAN_PURPOSE, NAME_CONTRACT_STATUS, DAYS_DECISION, NAME_PAYMENT_TYPE, CODE_REJECT_REASON, NAME_TYPE_SUITE, NAME_CLIENT_TYPE, NAME_GOODS_CATEGORY, NAME_PORTFOLIO, NAME_PRODUCT_TYPE, CHANNEL_TYPE, SELLERPLACE_AREA, NAME_SELLER_INDUSTRY, CNT_PAYMENT, NAME_YIELD_GROUP, PRODUCT_COMBINATION, DAYS_FIRST_DRAWING, DAYS_FIRST_DUE, DAYS_LAST_DUE_1ST_VERSION, DAYS_LAST_DUE, DAYS_TERMINATION, NFLAG_INSURED_ON_APPROVAL, AMT_ANNUITY_RANGE]
Index: []


In [20]:
df_pre_app[df_pre_app['AMT_APPLICATION'] > -1]['AMT_APPLICATION'].describe()

count    1.670214e+06
mean     1.752339e+05
std      2.927798e+05
min      0.000000e+00
25%      1.872000e+04
50%      7.104600e+04
75%      1.803600e+05
max      6.905160e+06
Name: AMT_APPLICATION, dtype: float64

In [21]:
#Define the bins
bins = [-1, 0, 5000, 10000, 15000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000,150000,200000,250000, float('inf')]
#Define labels for each bin
labels = ['Applied with Zero Amount', '0-5000',  '5000-10000', '10000-15000', '15000-20000', '20000-30000', '30000-40000', '40000-50000', '50000-60000', '60000-70000', '70000-80000',
          '80000-90000', '90000-100000', '100000-150000','150000-200000','200000-250000','250000+']
#Create a new column with bin labels
df_pre_app['AMT_APPLICATION_RANGE'] = pd.cut(df_pre_app['AMT_APPLICATION'], bins=bins, labels=labels)
#Display the counts in each bin
print(df_pre_app['AMT_APPLICATION_RANGE'].value_counts())

Applied with Zero Amount    392402
250000+                     306291
100000-150000               191269
40000-50000                 114851
200000-250000                93858
150000-200000                90729
20000-30000                  83248
30000-40000                  78498
80000-90000                  73653
60000-70000                  70063
50000-60000                  57858
70000-80000                  46867
90000-100000                 37020
15000-20000                  29338
10000-15000                   3984
5000-10000                     279
0-5000                           6
Name: AMT_APPLICATION_RANGE, dtype: int64


In [22]:
df_pre_app['AMT_CREDIT'].describe()

count    1.670214e+06
mean     1.961139e+05
std      3.185746e+05
min      0.000000e+00
25%      2.416050e+04
50%      8.054100e+04
75%      2.164185e+05
max      6.905160e+06
Name: AMT_CREDIT, dtype: float64

In [23]:
df_pre_app.loc[df_pre_app['NAME_CONTRACT_STATUS'] != 'Approved', 'AMT_CREDIT'] = -1
#Define the bins
bins = [-2, -0.5, 5000, 10000, 15000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000,150000,200000,250000, float('inf')]
#Define labels for each bin
labels = ['Applied but Not Approved', '0-5000',  '5000-10000', '10000-15000', '15000-20000', '20000-30000', '30000-40000', '40000-50000', '50000-60000', '60000-70000', '70000-80000',
          '80000-90000', '90000-100000', '100000-150000','150000-200000','200000-250000','250000+']
#Create a new column with bin labels
df_pre_app['AMT_CREDIT_RANGE'] = pd.cut(df_pre_app['AMT_CREDIT'], bins=bins, labels=labels)
#Display the counts in each bin
print(df_pre_app['AMT_CREDIT_RANGE'].value_counts())

Applied but Not Approved    633433
250000+                     230850
100000-150000               153815
40000-50000                  88387
150000-200000                85128
20000-30000                  76054
30000-40000                  67481
200000-250000                55962
50000-60000                  54940
60000-70000                  51021
80000-90000                  47599
70000-80000                  45112
90000-100000                 39566
15000-20000                  31103
10000-15000                   7934
0-5000                        1552
5000-10000                     277
Name: AMT_CREDIT_RANGE, dtype: int64


In [24]:
print(df_pre_app[df_pre_app['AMT_DOWN_PAYMENT'].isnull()])
df_pre_app['AMT_DOWN_PAYMENT'] = df_pre_app['AMT_DOWN_PAYMENT'].fillna(0)

         SK_ID_PREV  SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE WEEKDAY_APPR_PROCESS_START  HOUR_APPR_PROCESS_START FLAG_LAST_APPL_PER_CONTRACT  NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  RATE_INTEREST_PRIMARY  RATE_INTEREST_PRIVILEGED NAME_CASH_LOAN_PURPOSE NAME_CONTRACT_STATUS  DAYS_DECISION      NAME_PAYMENT_TYPE CODE_REJECT_REASON  NAME_TYPE_SUITE NAME_CLIENT_TYPE NAME_GOODS_CATEGORY NAME_PORTFOLIO NAME_PRODUCT_TYPE             CHANNEL_TYPE  SELLERPLACE_AREA NAME_SELLER_INDUSTRY  CNT_PAYMENT NAME_YIELD_GROUP  PRODUCT_COMBINATION  DAYS_FIRST_DRAWING  DAYS_FIRST_DUE  DAYS_LAST_DUE_1ST_VERSION  DAYS_LAST_DUE  DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL AMT_ANNUITY_RANGE AMT_APPLICATION_RANGE          AMT_CREDIT_RANGE
1           2802425      108129         Cash loans    25188.615         607500.0    679671.0               NaN         607500.0                   THURSDAY                       11                           Y 

In [25]:
#Creating Downpayment Percentage column
downpayment_percentage = (df_pre_app['AMT_DOWN_PAYMENT'] / df_pre_app['AMT_APPLICATION']) * 100
#Add the downpayment_percentage column to  DataFrame
df_pre_app['DOWNPAYMENT_PERCENTAGE'] = downpayment_percentage
#filling null values
df_pre_app['DOWNPAYMENT_PERCENTAGE'] = df_pre_app['DOWNPAYMENT_PERCENTAGE'].fillna(0)
df_pre_app.loc[df_pre_app['NAME_CONTRACT_STATUS'] != 'Approved', 'DOWNPAYMENT_PERCENTAGE'] = -1

In [26]:
df_pre_app[df_pre_app['DOWNPAYMENT_PERCENTAGE'] > 0]['DOWNPAYMENT_PERCENTAGE'].describe()

count    367800.000000
mean         14.737166
std          10.018379
min           0.000019
25%          10.000000
50%          10.005817
75%          19.992748
max          91.819699
Name: DOWNPAYMENT_PERCENTAGE, dtype: float64

In [27]:
#Define the bins
bins = [-2, -0.05, 2, 4, 6, 8, 10, 15, 20, 25, 30, 40, 50, 75, 100]
#Define labels for each bin
labels = ['Applied but Not Approved', '0-2',  '2-4', '4-6', '6-8', '8-10',
          '10-15', '15-20', '20-25', '25-30','30-40','40-50', '50-75',
          '75-100']
#Create a new column with bin labels
df_pre_app['DOWNPAYMENT_PERCENTAGE_RANGE'] = pd.cut(df_pre_app['DOWNPAYMENT_PERCENTAGE'], bins=bins, labels=labels)
#Display the counts in each bin
print(df_pre_app['DOWNPAYMENT_PERCENTAGE_RANGE'].value_counts())

0-2                         678852
Applied but Not Approved    633433
10-15                       146776
8-10                         87496
20-25                        34917
15-20                        32824
30-40                        16121
25-30                        11715
40-50                         9376
4-6                           8036
50-75                         5237
6-8                           4576
2-4                            693
75-100                         162
Name: DOWNPAYMENT_PERCENTAGE_RANGE, dtype: int64


In [28]:
df_pre_app['AMT_GOODS_PRICE']=df_pre_app['AMT_GOODS_PRICE'].fillna(0)

In [29]:
df_pre_app[['AMT_APPLICATION','AMT_GOODS_PRICE']].sample(n=10)

,AMT_APPLICATION,AMT_GOODS_PRICE
1027575,450000.0,450000.0
1085115,100035.0,100035.0
824840,25272.0,25272.0
673008,315000.0,315000.0
933312,0.0,0.0
945163,83700.0,83700.0
840120,0.0,0.0
1108016,112410.0,112410.0
1213804,0.0,0.0
1628900,83245.5,83245.5


In [30]:
df_pre_app[['RATE_INTEREST_PRIVILEGED','RATE_INTEREST_PRIMARY']].sample(n=10)

,RATE_INTEREST_PRIVILEGED,RATE_INTEREST_PRIMARY
926788,NaN,NaN
1129956,NaN,NaN
1319799,NaN,NaN
626019,NaN,NaN
1609342,NaN,NaN
1177997,NaN,NaN
435563,NaN,NaN
1165456,NaN,NaN
925553,NaN,NaN
110090,0.852537,0.19333


In [31]:
df_pre_app['PRIVILEGED_CUST'] = 'No'
rate_col=['RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']
for col in rate_col:
  df_pre_app[col]=df_pre_app[col].fillna(0)
# Update 'PRIVILEGED_CUST' to 'Yes' where 'RATE_INTEREST_PRIVILEGED' is greater than zero
df_pre_app.loc[df_pre_app['RATE_INTEREST_PRIVILEGED'] > 0, 'PRIVILEGED_CUST'] = 'Yes'

#Print the updated DataFrame to verify the changes
print(df_pre_app[['RATE_INTEREST_PRIVILEGED', 'PRIVILEGED_CUST']].head())

   RATE_INTEREST_PRIVILEGED PRIVILEGED_CUST
0                  0.867336             Yes
1                  0.000000              No
2                  0.000000              No
3                  0.000000              No
4                  0.000000              No


In [32]:
# Replace values in the 'NAME_CASH_LOAN_PURPOSE' column
df_pre_app['NAME_CASH_LOAN_PURPOSE'].replace({'XAP': 'Not Disclosed',
                                               'XNA': 'Not Available',
                                               'Refusal to name the goal': 'Refused Disclosure'},
                                              inplace=True)

Both 'XAP' and 'XNA' can indicate cases where the purpose of the cash loan is not specified or not provided, 'XAP' might denote cases where the loan application is still relevant or applicable(Loan is not disclosed or not relevant to the dataset), whereas 'XNA' might suggest that the loan application itself is not applicable or not included in the dataset possibly because it's missing or incomplete..

In [33]:
df_pre_app['DAYS_DECISION']=abs(df_pre_app['DAYS_DECISION'])

In [34]:
print(df_pre_app['DAYS_DECISION'].sample(n=10))
df_pre_app['DAYS_DECISION'].describe()

944496      515
465721      299
1433724    2501
994084      438
1258963    1743
1088319     972
604024      587
72358      1233
246709     2855
1298986     311
Name: DAYS_DECISION, dtype: int64


count    1.670214e+06
mean     8.806797e+02
std      7.790997e+02
min      1.000000e+00
25%      2.800000e+02
50%      5.810000e+02
75%      1.300000e+03
max      2.922000e+03
Name: DAYS_DECISION, dtype: float64

In [35]:
#Define the bin edges
bin_edges = [0, 180, 365, 730, 1095, 1460, 1825,float('inf')]

#Define the bin labels
bin_labels = ['Below 6 months', 'Below 1 year', 'Below 2 years', 'Below 3 years', 'Below 4 years', 'Below 5 years', 'Above 5 years']

#Bin the 'DAYS_DECISION' column
df_pre_app['DAYS_DECISION_RANGE'] = pd.cut(df_pre_app['DAYS_DECISION'], bins=bin_edges, labels=bin_labels)

#Display the binned column
print(df_pre_app['DAYS_DECISION_RANGE'].value_counts())

Below 2 years     385098
Below 1 year      356896
Above 5 years     262634
Below 6 months    216844
Below 3 years     214716
Below 4 years     131666
Below 5 years     102360
Name: DAYS_DECISION_RANGE, dtype: int64


In [36]:
df_pre_app['NAME_TYPE_SUITE']=df_pre_app['NAME_TYPE_SUITE'].fillna('Unaccompanied')

In [37]:
df_pre_app['SELLERPLACE_AREA']=df_pre_app['SELLERPLACE_AREA'].fillna(0)
print(df_pre_app['SELLERPLACE_AREA'].sample(n=10))
df_pre_app['SELLERPLACE_AREA'].describe()

35038        -1
238883       72
1341845      -1
203590     1800
820546       -1
111900       42
1084378      -1
1471938      -1
1257002    1500
1017137    1880
Name: SELLERPLACE_AREA, dtype: int64


count    1.670214e+06
mean     3.139511e+02
std      7.127443e+03
min     -1.000000e+00
25%     -1.000000e+00
50%      3.000000e+00
75%      8.200000e+01
max      4.000000e+06
Name: SELLERPLACE_AREA, dtype: float64

In [38]:
# Define the bin edges
bin_edges = [-2, 0.2, 100, 500, 1000, 5000, float('inf')]

# Define the bin labels
bin_labels = ['Unknown', 'Small', 'Medium', 'Large', 'Very Large', 'Huge']

# Bin the 'SELLERPLACE_AREA' column
df_pre_app['SELLERPLACE_AREA_RANGE'] = pd.cut(df_pre_app['SELLERPLACE_AREA'], bins=bin_edges, labels=bin_labels)

# Display the binned column
print(df_pre_app['SELLERPLACE_AREA_RANGE'].value_counts())

Unknown       823198
Small         477939
Medium        167611
Very Large    142778
Large          52273
Huge            6415
Name: SELLERPLACE_AREA_RANGE, dtype: int64


In [39]:
df_pre_app['CNT_PAYMENT']=df_pre_app['CNT_PAYMENT'].fillna(0)
print(df_pre_app['CNT_PAYMENT'].sample(n=10))
df_pre_app['CNT_PAYMENT'].describe()

161614     12.0
1626146    10.0
485751      6.0
1074282    12.0
511787     10.0
242293      6.0
55582       0.0
869859     48.0
1430016     0.0
1232850    12.0
Name: CNT_PAYMENT, dtype: float64


count    1.670214e+06
mean     1.247621e+01
std      1.447588e+01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+01
75%      1.600000e+01
max      8.400000e+01
Name: CNT_PAYMENT, dtype: float64

In [40]:

# Define the bin edges
bin_edges = [-1, 0, 3, 6, 9, 12, 15, 18, 21, 24, 30, 36, 48, 60, float('inf')]

# Define the bin labels
bin_labels = ['Unknown', '1-3', '3-6', '6-9', '9-12','12-15','15-18','18-21','21-24',
              '24-30','30-36','36-48','48-60','60 Above']

# Bin the 'CNT_PAYMENT' column
df_pre_app['CNT_PAYMENT_RANGE'] = pd.cut(df_pre_app['CNT_PAYMENT'], bins=bin_edges, labels=bin_labels)

# Display the binned column
print(df_pre_app['CNT_PAYMENT_RANGE'].value_counts())

Unknown     517215
9-12        465569
3-6         221342
21-24       137828
15-18        83188
30-36        72605
48-60        55709
36-48        54470
6-9          33019
24-30        16953
12-15         9208
18-21         1814
1-3           1100
60 Above       194
Name: CNT_PAYMENT_RANGE, dtype: int64


In [41]:
df_pre_app['PRODUCT_COMBINATION']=df_pre_app['PRODUCT_COMBINATION'].fillna('Other')

In [42]:
fill_col=['DAYS_FIRST_DRAWING','DAYS_FIRST_DUE',
          'DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION','NFLAG_INSURED_ON_APPROVAL' ]
for col in fill_col:
  df_pre_app[col]=df_pre_app[col].fillna(0)

In [43]:
days_col=['DAYS_FIRST_DRAWING','DAYS_FIRST_DUE',
          'DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION']
for col in days_col:
  df_pre_app[col]=abs(df_pre_app[col])
df_pre_app[['DAYS_FIRST_DRAWING','DAYS_FIRST_DUE',
          'DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION']].describe()

,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
count,1.670214e+06,1.670214e+06,1.670214e+06,1.670214e+06,1.670214e+06
mean,2.043834e+05,9.521985e+03,2.099319e+04,4.665864e+04,4.985451e+04
std,1.812813e+05,5.618513e+04,8.400628e+04,1.212198e+05,1.247367e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.652430e+05,3.360000e+02,1.540000e+02,2.920000e+02,2.830000e+02
75%,3.652430e+05,1.132000e+03,9.960000e+02,1.518000e+03,1.544000e+03
max,3.652430e+05,3.652430e+05,3.652430e+05,3.652430e+05,3.652430e+05


In [44]:
# Define bin edges for days
bin_edges_days = [-1, 0, 30, 90, 180, 365,730, float('inf')]

# Define bin labels for days
bin_labels_days = ['Unknown', 'Within 1 month', 'Within 3 months', 'Within 6 months', 'Within 1 year', 'Within 2 year','More than 2 year']

# Bin the columns
for column in days_col:
    df_pre_app[f'{column}_RANGE'] = pd.cut(df_pre_app[column], bins=bin_edges_days, labels=bin_labels_days)

# Display binned columns
for column in days_col:
    print(df_pre_app[ f'{column}_RANGE'].value_counts())


More than 2 year    962432
Unknown             673065
Within 1 year        16182
Within 2 year        14464
Within 6 months       2601
Within 3 months        911
Within 1 month         559
Name: DAYS_FIRST_DRAWING_RANGE, dtype: int64
Unknown             673065
More than 2 year    591546
Within 2 year       224998
Within 1 year       117457
Within 6 months      41586
Within 3 months      16068
Within 1 month        5494
Name: DAYS_FIRST_DUE_RANGE, dtype: int64
Unknown             673770
More than 2 year    502433
Within 2 year       167199
Within 1 year       141532
Within 6 months      85513
Within 3 months      63376
Within 1 month       36391
Name: DAYS_LAST_DUE_1ST_VERSION_RANGE, dtype: int64
Unknown             673065
More than 2 year    630279
Within 2 year       163401
Within 1 year       106480
Within 6 months      50307
Within 3 months      31691
Within 1 month       14991
Name: DAYS_LAST_DUE_RANGE, dtype: int64
Unknown             673065
More than 2 year    629816
Within 2 yea

In [45]:
#Display descriptive statistics for categorical (object) columns in the DataFrame
df_pre_app.isnull().sum()

SK_ID_PREV                         0
SK_ID_CURR                         0
NAME_CONTRACT_TYPE                 0
AMT_ANNUITY                        0
AMT_APPLICATION                    0
AMT_CREDIT                         0
AMT_DOWN_PAYMENT                   0
AMT_GOODS_PRICE                    0
WEEKDAY_APPR_PROCESS_START         0
HOUR_APPR_PROCESS_START            0
FLAG_LAST_APPL_PER_CONTRACT        0
NFLAG_LAST_APPL_IN_DAY             0
RATE_DOWN_PAYMENT                  0
RATE_INTEREST_PRIMARY              0
RATE_INTEREST_PRIVILEGED           0
NAME_CASH_LOAN_PURPOSE             0
NAME_CONTRACT_STATUS               0
DAYS_DECISION                      0
NAME_PAYMENT_TYPE                  0
CODE_REJECT_REASON                 0
NAME_TYPE_SUITE                    0
NAME_CLIENT_TYPE                   0
NAME_GOODS_CATEGORY                0
NAME_PORTFOLIO                     0
NAME_PRODUCT_TYPE                  0
CHANNEL_TYPE                       0
SELLERPLACE_AREA                   0
N

In [46]:
df_pre_app[df_pre_app.duplicated()]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,AMT_ANNUITY_RANGE,AMT_APPLICATION_RANGE,AMT_CREDIT_RANGE,DOWNPAYMENT_PERCENTAGE,DOWNPAYMENT_PERCENTAGE_RANGE,PRIVILEGED_CUST,DAYS_DECISION_RANGE,SELLERPLACE_AREA_RANGE,CNT_PAYMENT_RANGE,DAYS_FIRST_DRAWING_RANGE,DAYS_FIRST_DUE_RANGE,DAYS_LAST_DUE_1ST_VERSION_RANGE,DAYS_LAST_DUE_RANGE,DAYS_TERMINATION_RANGE


In [47]:
df_pre_app = df_pre_app.drop_duplicates()
df_pre_app.shape

(1670214, 51)